# Algoritm

In [64]:
# Initiate environment

%matplotlib inline

import pandas as pd
#import DataFrame as df

import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

%matplotlib inline

In [65]:
# Constants

# Number of clusters
number_of_clusteroids = 10

year_slice = 2016

num_of_rows = 200

# Columns to compare
columns_to_count_distance=[
    "region_client",
    "region_sc",
    "product_name",
    "product",
    "tarif_name",
    "price_item_name",
    "price_set_name",
    "month_length",
    "is_first",
    "Cnt",
]

# Clusteroid ID field
clusteroid_id_field = "id_org"
cluster_field = "Cluster"
cluster_distance_to_clusteroid_field = "Cluster_distance"


In [66]:
# Read the Data

# Todo, slice years

dateparser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

df_data = pd.read_csv("uc_kep_obs/pays.txt",
                 delimiter="\t", 
                 encoding="utf-8",
                 nrows=num_of_rows, 
                 parse_dates=[11,12], 
                 infer_datetime_format = True,
                 date_parser=dateparser
                )
# df
df_data = df_data[df_data['BDate'].dt.year == year_slice]

In [67]:
# Add colunm for Cluster number

# it is not working
#cols_dtype={cluster_field : str,
#            cluster_distance_to_clusteroid_field : np.int}

#df_clusters_info = pd.DataFrame(dtype=cols_dtype)


df_data[cluster_field] = "0"
#D19CA103-3DFE-411C-B52A-32C12E6FDB4F"
df_data[cluster_distance_to_clusteroid_field] = np.nan

In [68]:
%matplotlib inline

In [69]:
# N random clusteroids

clusteroids = df_data.sample(number_of_clusteroids)[clusteroid_id_field].index.values

clusteroids

array([ 18,  52,  38, 149, 167, 158, 116, 154,  53, 183])

In [70]:
#df_data

In [71]:
log_value =[]

for point in df_data.index: # Iterete through data to clusterise
    for clusteroid in clusteroids: # iterate throught clusters
        log_value.append(clusteroid)
#log_value

In [72]:
# Assign to clusters

# Todo: Add converging, isChanged
log_value =[]

for point in df_data.index.values: # Iterete through data to clusterise
    min_distance = len(columns_to_count_distance)
    min_clusteroid = -1
    for clusteroid in clusteroids: # iterate throught clusters
        row1 = df_data.loc[point][columns_to_count_distance]
        row2 = df_data.loc[clusteroid][columns_to_count_distance]
        unique_columns = row1 == row2
        lenght = len(columns_to_count_distance)
        unique_columns_sum = unique_columns.sum()
        distance = lenght - unique_columns_sum
        if distance < min_distance:
            min_distance = distance
            min_clusteroid = clusteroid #df_data.loc[clusteroid][clusteroid_id_field].index.values
            #log_value.append("new min distance {} clusteroid {}".format(min_distance,min_clusteroid))
    df_data.set_value(point,cluster_field,min_clusteroid)
    df_data.set_value(point,cluster_distance_to_clusteroid_field,min_distance)
    print ("Point {} assignbed to {} with distance {}".format(point, min_clusteroid, min_distance))

#log_value

In [83]:
df_data[cluster_field].unique()

array([149, 61, 158, 183, 167, 22, 27, 53, 116, 154], dtype=object)

In [75]:
#df_data

In [76]:
clusteroids

array([ 18,  52,  38, 149, 167, 158, 116, 154,  53, 183])

In [52]:
# Recalculate clusteriods

# Get old clusteroid, for each element in the cluster calculate sum of distances to all the other points in cluster. 
# Keep Min -> save as new clusteroid

log_value =[]

old_clusteroids = np.copy(clusteroids)
print ""
print "---------"
print "Old Clusters {}".format(old_clusteroids)
print "Clusters {}".format(clusteroids)

old_clusteroids = np.copy(clusteroids)
clusteroids = []

for clusteroid in old_clusteroids: # Iterete through to clusteroids
    min_distance = df_data[cluster_distance_to_clusteroid_field].sum()
    min_clusteroid = clusteroid
    print ("Cluster {} with distance {}".format(min_clusteroid, min_distance))
    for element_i in df_data[df_data[cluster_field] == clusteroid].index.values: # iterate throught cluster
        print ("--Element_i {}".format(element_i))
        sum_distance = len(columns_to_count_distance)
        sum_clusteroid = -1
        for element_j in df_data[df_data[cluster_field] == clusteroid].index.values: # iterate throught cluster
            #print ("----Element_j {}".format(element_j))
            row1 = df_data.loc[element_i][columns_to_count_distance]
            row2 = df_data.loc[element_j][columns_to_count_distance]
            unique_columns = row1 == row2
            lenght = len(columns_to_count_distance)
            unique_columns_sum = unique_columns.sum()
            distance = lenght - unique_columns_sum
            sum_distance = sum_distance + distance
            #print sum_distance
        if sum_distance <= min_distance:
            min_distance = sum_distance
            min_clusteroid = element_i
            #log_value.append("new clusteroid {} min summ distance {}".format(element_i,sum_distance))
            print ("----New clusteroid {} min summ distance {}".format(element_i,sum_distance))
    #clusteroids.append(min_clusteroid)
    if min_clusteroid != -1:
        print ("Clusteroid updated {} by {}".format(clusteroid,min_clusteroid))
        clusteroids.append(min_clusteroid)
    else: 
        clusteroids.append(clusteroid)
        #df_data = df_data[df_data['BDate'].dt.year == 2016]
        #clusteroids = np.append(clusteroids, df_data[df_data[clusteroid_id_field] == min_clusteroid]["id_org"].index.values[0])
        #print ("Clusteroid {}".format(clusteroid))
        #print ("element_i {} element_j {}".format(element_i,element_j))

#clusteroids = np.copy(old_clusteroids)
print "Old Clusters {}".format(old_clusteroids)
print "Clusters {}".format(clusteroids)
print "---------"
print ""

In [86]:
# Recalculate clusteriods 2

# Get old clusteroid, for each element in the cluster calculate sum of distances to all the other points in cluster. 
# Keep Min -> save as new clusteroid

log_value =[]

old_clusteroids = np.copy(clusteroids)
print ""
print "---------"
#print "Old Clusters {}".format(old_clusteroids)
print "Clusters {}".format(clusteroids)

old_clusteroids = np.copy(clusteroids)
clusteroids = []

for clusteroid in old_clusteroids: # Iterete through to clusteroids
    print ("Cluster {}".format(clusteroid))
    min_distance = df_data[df_data[cluster_field] == clusteroid][cluster_distance_to_clusteroid_field].sum()
    min_clusteroid = clusteroid
    for element_i in df_data[df_data[cluster_field] == clusteroid].index.values: # iterate throught cluster
        #print ("--Element_i {}".format(element_i))
        sum_distance = 0
        sum_clusteroid = clusteroid
        for element_j in df_data[df_data[cluster_field] == clusteroid].index.values: # iterate throught cluster
            #print ("----Element_j {}".format(element_j))
            row1 = df_data.loc[element_i][columns_to_count_distance]
            row2 = df_data.loc[element_j][columns_to_count_distance]
            unique_columns = row1 == row2
            lenght = len(columns_to_count_distance)
            unique_columns_sum = unique_columns.sum()
            distance = lenght - unique_columns_sum
            sum_distance = sum_distance + distance
            #print sum_distance
        if sum_distance < min_distance:
            min_distance = sum_distance
            min_clusteroid = element_i
            #log_value.append("new clusteroid {} min summ distance {}".format(element_i,sum_distance))
            print ("----New clusteroid {} min summ distance {}".format(element_i,sum_distance))
    #clusteroids.append(min_clusteroid)
    clusteroids.append(min_clusteroid)
    #if min_clusteroid != -1:
        #print ("Clusteroid updated {} by {}".format(clusteroid,min_clusteroid))
        #clusteroids.append(min_clusteroid)
        #df_data = df_data[df_data['BDate'].dt.year == 2016]
        #clusteroids = np.append(clusteroids, df_data[df_data[clusteroid_id_field] == min_clusteroid]["id_org"].index.values[0])
        #print ("Clusteroid {}".format(clusteroid))
        #print ("element_i {} element_j {}".format(element_i,element_j))

#clusteroids = np.copy(old_clusteroids)
print "Old Clusters {}".format(old_clusteroids)
print "Clusters {}".format(clusteroids)
print "---------"
print ""

In [87]:
# Pere assign clusters

# Todo: Add converging, isChanged

for point in df_data.index.values: # Iterete through data to clusterise
    min_distance = len(columns_to_count_distance)
    min_clusteroid = -1
    for clusteroid in clusteroids: # iterate throught clusters
        row1 = df_data.loc[point][columns_to_count_distance]
        row2 = df_data.loc[clusteroid][columns_to_count_distance]
        unique_columns = row1 == row2
        lenght = len(columns_to_count_distance)
        unique_columns_sum = unique_columns.sum()
        distance = lenght - unique_columns_sum
        if distance < min_distance:
            min_distance = distance
            min_clusteroid = clusteroid #df_data.loc[clusteroid][clusteroid_id_field].index.values
            #log_value.append("new min distance {} clusteroid {}".format(min_distance,min_clusteroid))
    df_data.set_value(point,cluster_field,min_clusteroid)
    df_data.set_value(point,cluster_distance_to_clusteroid_field,min_distance)
    print ("Point {} assignbed to {} with distance {}".format(point, min_clusteroid, min_distance))

#log_value

In [31]:
#df_data.sort(cluster_field, ascending=True)
#df.sort('Bytes', ascending=False)
df_data[cluster_field].unique()

array([197, 61, 57, 12, 103, 181, 38, 155, 55, 132], dtype=object)

In [32]:
old_clusteroids

array([181,  61, 103, 155,  55,  38,  57, 132, 197,  12])

In [80]:
clusteroids

[61, 22, 27, 149, 167, 158, 116, 154, 53, 183]

In [55]:
clusteroid = "DE7EBBBD-A930-4189-A9DD-E04BA04466FF"
# df_data[df_data[cluster_field] == clusteroid].index
min_clusteroid = "9EF648A2-363B-4D85-AF81-3EC1423EF8C6"
df_data[df_data[clusteroid_id_field] == min_clusteroid]["id_org"].index.values[0]

787

In [77]:
df_data[df_data[cluster_field] == 18][cluster_distance_to_clusteroid_field].sum()

29.0

In [176]:
clusteroid = 214
df_data[df_data[cluster_field] == clusteroid].index.values

array([], dtype=int64)

In [78]:
df_data[df_data[cluster_field] == 18].index.values

array([  3,  18,  42,  47,  61,  62,  84, 123, 127, 144, 181, 196])

In [73]:
df_data.to_csv("temp_1_clustering.csv",index=True, encoding="utf-8")

In [82]:
df_data.to_csv("temp_2_clustering.csv",index=True, encoding="utf-8")

In [88]:
df_data.to_csv("temp_last_clustering.csv",index=True, encoding="utf-8")